In [1]:
from tqdm import tqdm

import json
import pandas as pd

from polyglot.text import Text, Word

from polyglot.detect.base import logger
logger.disabled = True
import pycld2

# Load JSON

In [2]:
with open("ocr.json") as handle:
    raw = handle.read()
    
data = json.loads(raw)

data = data

In [3]:
data[1][2]

{'text': {'title': 'Geloovige Christenen in Engeland.',
  'p': ["Begrijpelijkerwijze wekt het verbazing, dat zooveel geloovige Christenen in Engeland met het hartstochtelijk Jingoïsme eener volkswaan-streelende Pers in koor meezingen, en dat Steads consciëntievraag: Sliall I slay my brother boer?geen weerklank aan de leiders der Christelijke actie in Engeland ontlokt heeft. Ter verklaring hiervan dient rekening gehouden met wat Edmund Burke reeds gispte in de Aboriginal Societies, en niet minder met hetgeen men thans noemen durft: Englaniïs Christ. Over beide ter toelichting een kort woord. Op het laatst der vorige eeuw was de «fatsoenlijke klasse» der maatschappij, vooral op het Continent, maar ten deele ook in Engeland, sociaal en moreel bedorven. De vloek der toenmalige Fransche en Duitsche hoven was allengs in alle aristocratische kringen geslopen ; en vandaar naar de kringen der bemiddelden afgedaald. De wezenlijk «rotte» literatuur dier dagen was van dat droef verschijnsel én vru

# Transform JSON into DataFrame

In [4]:
def get_row(row, index):
    link, meta_d, text_d = row
    yield ("link", link)#, *meta_d.items(), text_d["text"].items()
    yield from meta_d.items()
    yield from text_d["text"].items()
    yield "original_i", index
        
def restructure(dat):
    for i, row in enumerate(dat): yield dict(get_row(row, i))
        
def separate_paragraphs(rows):
    for row_dict in tqdm(rows):
        p = row_dict["p"]
        if isinstance(p, str):
            dict_copy = {k: v for k, v in row_dict.items()}
            dict_copy["paragraph_i"] = 0
            dict_copy["ps"] = p
            yield dict_copy
        elif isinstance(p, list):
            for i, par in enumerate(p):
                dict_copy = {k: v for k, v in row_dict.items()}
                dict_copy["paragraph_i"] = i
                dict_copy["ps"] = par
#                 print()
                yield dict_copy
     
def json_to_sent_df(data):
    return pd.DataFrame.from_records(separate_paragraphs(restructure(data)))

In [5]:
df = json_to_sent_df(data)

56748it [00:01, 30137.19it/s]


In [6]:
df

,link,europeana_issue_id,kb_issue_id,oai_issue_id,matching_queries,kb_oai_metadata_queried,kb_oai_metadata,title,p,original_i,paragraph_i,ps
0,http://resolver.kb.nl/resolve?urn=ddd:11054039...,/9200359/BibliographicResource_3000115832584,ddd:110540397:mpeg21,DDD:ddd:110540397:mpeg21,"[aboriginal AND date within ""1890-01-01 1899-1...",True,"{'datestamp': '2021-01-06T07:18:08.959Z', 'tit...",Cricket.,•i eigenaardig geval deed zich voor in en trüd...,0,0,•i eigenaardig geval deed zich voor in en trüd...
1,http://resolver.kb.nl/resolve?urn=ddd:01113589...,/9200359/BibliographicResource_3000115733752,ddd:011135894:mpeg21,DDD:ddd:011135894:mpeg21,"[aboriginal AND date within ""1890-01-01 1899-1...",True,"{'datestamp': '2021-01-06T03:35:49.005Z', 'tit...",Geloovige Christenen in Engeland.,"[Begrijpelijkerwijze wekt het verbazing, dat z...",1,0,"Begrijpelijkerwijze wekt het verbazing, dat zo..."
2,http://resolver.kb.nl/resolve?urn=ddd:01113589...,/9200359/BibliographicResource_3000115733752,ddd:011135894:mpeg21,DDD:ddd:011135894:mpeg21,"[aboriginal AND date within ""1890-01-01 1899-1...",True,"{'datestamp': '2021-01-06T03:35:49.005Z', 'tit...",Geloovige Christenen in Engeland.,"[Begrijpelijkerwijze wekt het verbazing, dat z...",1,1,Missie en koloniale veroveringspolitiek zijn i...
3,http://resolver.kb.nl/resolve?urn=ddd:01103192...,/9200359/BibliographicResource_3000115594603,ddd:011031929:mpeg21,DDD:ddd:011031929:mpeg21,"[aboriginal AND date within ""1890-01-01 1899-1...",True,"{'datestamp': '2021-01-05T20:48:21.456Z', 'tit...",INGEZONDEN. KNOWLEDGE IS POWER.,Mr. Zorabs aphorism is not quite up to date. I...,2,0,Mr. Zorabs aphorism is not quite up to date. I...
4,http://resolver.kb.nl/resolve?urn=ddd:11061786...,/9200359/BibliographicResource_3000115938711,ddd:110617863:mpeg21,DDD:ddd:110617863:mpeg21,"[aboriginal AND date within ""1890-01-01 1899-1...",True,"{'datestamp': '2021-01-06T12:45:25.682Z', 'tit...",FEUILLETON. VAN OVERAL NAAR OVERAL EN VICE-VER...,[En die geluiden die men hoort. Zij schijcen b...,3,0,En die geluiden die men hoort. Zij schijcen bo...
...,...,...,...,...,...,...,...,...,...,...,...,...
364669,http://resolver.kb.nl/resolve?urn=ddd:01031358...,/9200359/BibliographicResource_3000112938684,ddd:010313587:mpeg21,DDD:ddd:010313587:mpeg21,"[zwartje AND date within ""1940-01-01 1941-12-3...",True,"{'datestamp': '2021-01-06T23:08:30.287Z', 'tit...",Onze eigen KRANT,[Verantwoording VAN INO£KOM€N WERK JEen opstel...,56747,25,DRAAIENDE WIELEN EN EEN SCHRIKACHTIG PAARD. Bo...
364670,http://resolver.kb.nl/resolve?urn=ddd:01031358...,/9200359/BibliographicResource_3000112938684,ddd:010313587:mpeg21,DDD:ddd:010313587:mpeg21,"[zwartje AND date within ""1940-01-01 1941-12-3...",True,"{'datestamp': '2021-01-06T23:08:30.287Z', 'tit...",Onze eigen KRANT,[Verantwoording VAN INO£KOM€N WERK JEen opstel...,56747,26,er geen water in de buurt was namen we ieder m...
364671,http://resolver.kb.nl/resolve?urn=ddd:01031358...,/9200359/BibliographicResource_3000112938684,ddd:010313587:mpeg21,DDD:ddd:010313587:mpeg21,"[zwartje AND date within ""1940-01-01 1941-12-3...",True,"{'datestamp': '2021-01-06T23:08:30.287Z', 'tit...",Onze eigen KRANT,[Verantwoording VAN INO£KOM€N WERK JEen opstel...,56747,27,A. T. (l« j.) Twis»
364672,http://resolver.kb.nl/resolve?urn=ddd:01031358...,/9200359/BibliographicResource_3000112938684,ddd:010313587:mpeg21,DDD:ddd:010313587:mpeg21,"[zwartje AND date within ""1940-01-01 1941-12-3...",True,"{'datestamp': '2021-01-06T23:08:30.287Z', 'tit...",Onze eigen KRANT,[Verantwoording VAN INO£KOM€N WERK JEen opstel...,56747,28,A. W. (11 j.) Hintham


# Sentence Segmentation

In [7]:
def tokenize(par):
    try:
        return Text(par, hint_language_code="nl").sentences
    except pycld2.error:
        return [par]



def sent_iter(df):
    for j, row in tqdm(df.iterrows()):
        sents = tokenize(row["ps"])
        for i, s in enumerate(sents):
            copy = {k: v for k, v in row.items()}
            copy["s"] = str(s)
            copy["sentence_i"] = i
            yield copy
            
df_sents = pd.DataFrame.from_records(sent_iter(df))

364674it [02:49, 2146.80it/s]


In [8]:
df_sents["typicality"] = [0.]*df_sents.shape[0]

In [9]:
df_sents

,link,europeana_issue_id,kb_issue_id,oai_issue_id,matching_queries,kb_oai_metadata_queried,kb_oai_metadata,title,p,original_i,paragraph_i,ps,s,sentence_i,typicality
0,http://resolver.kb.nl/resolve?urn=ddd:11054039...,/9200359/BibliographicResource_3000115832584,ddd:110540397:mpeg21,DDD:ddd:110540397:mpeg21,"[aboriginal AND date within ""1890-01-01 1899-1...",True,"{'datestamp': '2021-01-06T07:18:08.959Z', 'tit...",Cricket.,•i eigenaardig geval deed zich voor in en trüd...,0,0,•i eigenaardig geval deed zich voor in en trüd...,•i eigenaardig geval deed zich voor in en trüd...,0,0.0
1,http://resolver.kb.nl/resolve?urn=ddd:11054039...,/9200359/BibliographicResource_3000115832584,ddd:110540397:mpeg21,DDD:ddd:110540397:mpeg21,"[aboriginal AND date within ""1890-01-01 1899-1...",True,"{'datestamp': '2021-01-06T07:18:08.959Z', 'tit...",Cricket.,•i eigenaardig geval deed zich voor in en trüd...,0,0,•i eigenaardig geval deed zich voor in en trüd...,De -ast Lancaahire m aiir-'h 9 wtokets voor 70...,1,0.0
2,http://resolver.kb.nl/resolve?urn=ddd:11054039...,/9200359/BibliographicResource_3000115832584,ddd:110540397:mpeg21,DDD:ddd:110540397:mpeg21,"[aboriginal AND date within ""1890-01-01 1899-1...",True,"{'datestamp': '2021-01-06T07:18:08.959Z', 'tit...",Cricket.,•i eigenaardig geval deed zich voor in en trüd...,0,0,•i eigenaardig geval deed zich voor in en trüd...,Uia iieb dat de batsman gerechtigd was do laat...,2,0.0
3,http://resolver.kb.nl/resolve?urn=ddd:11054039...,/9200359/BibliographicResource_3000115832584,ddd:110540397:mpeg21,DDD:ddd:110540397:mpeg21,"[aboriginal AND date within ""1890-01-01 1899-1...",True,"{'datestamp': '2021-01-06T07:18:08.959Z', 'tit...",Cricket.,•i eigenaardig geval deed zich voor in en trüd...,0,0,•i eigenaardig geval deed zich voor in en trüd...,"* * * De moest bekende, rocords op het gobio I...",3,0.0
4,http://resolver.kb.nl/resolve?urn=ddd:11054039...,/9200359/BibliographicResource_3000115832584,ddd:110540397:mpeg21,DDD:ddd:110540397:mpeg21,"[aboriginal AND date within ""1890-01-01 1899-1...",True,"{'datestamp': '2021-01-06T07:18:08.959Z', 'tit...",Cricket.,•i eigenaardig geval deed zich voor in en trüd...,0,0,•i eigenaardig geval deed zich voor in en trüd...,(De Athkit.}.,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4223979,http://resolver.kb.nl/resolve?urn=ddd:01031358...,/9200359/BibliographicResource_3000112938684,ddd:010313587:mpeg21,DDD:ddd:010313587:mpeg21,"[zwartje AND date within ""1940-01-01 1941-12-3...",True,"{'datestamp': '2021-01-06T23:08:30.287Z', 'tit...",Onze eigen KRANT,[Verantwoording VAN INO£KOM€N WERK JEen opstel...,56747,27,A. T. (l« j.) Twis»,Twis»,2,0.0
4223980,http://resolver.kb.nl/resolve?urn=ddd:01031358...,/9200359/BibliographicResource_3000112938684,ddd:010313587:mpeg21,DDD:ddd:010313587:mpeg21,"[zwartje AND date within ""1940-01-01 1941-12-3...",True,"{'datestamp': '2021-01-06T23:08:30.287Z', 'tit...",Onze eigen KRANT,[Verantwoording VAN INO£KOM€N WERK JEen opstel...,56747,28,A. W. (11 j.) Hintham,A.,0,0.0
4223981,http://resolver.kb.nl/resolve?urn=ddd:01031358...,/9200359/BibliographicResource_3000112938684,ddd:010313587:mpeg21,DDD:ddd:010313587:mpeg21,"[zwartje AND date within ""1940-01-01 1941-12-3...",True,"{'datestamp': '2021-01-06T23:08:30.287Z', 'tit...",Onze eigen KRANT,[Verantwoording VAN INO£KOM€N WERK JEen opstel...,56747,28,A. W. (11 j.) Hintham,W. (11 j.),1,0.0
4223982,http://resolver.kb.nl/resolve?urn=ddd:01031358...,/9200359/BibliographicResource_3000112938684,ddd:010313587:mpeg21,DDD:ddd:010313587:mpeg21,"[zwartje AND date within ""1940-01-01 1941-12-3...",True,"{'datestamp': '2021-01-06T23:08:30.287Z', 'tit...",Onze eigen KRANT,[Verantwoording VAN INO£KOM€N WERK JEen opstel...,56747,28,A. W. (11 j.) Hintham,Hintham,2,0.0


In [ ]:
import matplotlib.pyplot as plt


lens = [x.shape[0] for _, x in df_sents.groupby("original_i")]
_ = plt.hist(lens, bins=1000)
plt.xlim((0, 100))

# Transform back to JSON

In [26]:
import numpy

meta_keys = {'europeana_issue_id', 'kb_issue_id', 'oai_issue_id', 'matching_queries', 'kb_oai_metadata_queried', 'kb_oai_metadata'}


# add `original_id` to first DF (repeat)
# -> repeat in second DF
# -> use to sort when transforming back (sort by ['original_id', 'paragraph_id', 'sentence_id'])

sorted_df = df_sents.sort_values(["original_i", "paragraph_i", "sentence_i"],
                                kind="mergesort")


samples = []

for original_ind, small_df in tqdm(df_sents.groupby("link")):

    row0 = small_df.iloc[0]

    sentences = list(small_df.s)
    typicalities = list(small_df.typicality)

    if not pd.Index(small_df["paragraph_i"]).is_monotonic:
        print(small_df.original_i, small_df["paragraph_i"])
        
#     for paragraph_i, tiny_df in small_df.groupby("paragraph_i"):
#         if not pd.Index(tiny_df["sentence_i"]).is_monotonic:
#             print(small_df.original_i, small_df["sentenceparagraph_i"])
    
    
    
    text_d = {"text":
             {"title": row0["title"],
             "p": row0["p"],
             "s": sentences,
             "typicality": typicalities}} 
    
    
    sample_ls = [row0["link"],
                {k: (row0[k] if not (type(row0[k]) is numpy.bool_) else bool(row0[k])) for k in meta_keys},
                text_d]

    samples.append(sample_ls)
    

100%|██████████| 56748/56748 [00:40<00:00, 1398.20it/s]


In [27]:
with open("valentin_ocr.json", "w") as handle:
     json.dump(samples, handle, indent=4, ensure_ascii=False)

In [12]:
sample_ls

['http://resolver.kb.nl/resolve?urn=ddd:110638029:mpeg21:a0002:ocr',
 {'matching_queries': ['islamiet AND date within "1930-01-01 1939-12-31" AND type=artikel'],
  'kb_oai_metadata_queried': True,
  'oai_issue_id': 'DDD:ddd:110638029:mpeg21',
  'europeana_issue_id': '/9200359/BibliographicResource_3000115940709',
  'kb_issue_id': 'ddd:110638029:mpeg21',
  'kb_oai_metadata': {'datestamp': '2021-01-06T12:50:00.239Z',
   'title': 'Soerabaijasch handelsblad',
   'date': '1930-12-24',
   'temporal': 'Soerabaijasch handelsblad',
   'publisher': 'Kolff & Co',
   'spatial_distribution': 'Nederlands-Indië / Indonesië',
   'spatial_origin': 'Soerabaja',
   'language': ['nl']}},
 {'text': {'title': 'Het land der twee rivieren VII. door H. C. Zentgraaff.',
   'p': 'In Mosoel mengen de eeuwen zich in al het zichtbare en onzichtbare leven: het materieele en het geestelijke. Het verleden handhaaft er zich in kleeding en gebruik, in taal en zede. Bij de oude poort Bab el Tob krioelt het leven van eeuw

In [24]:
type(sorted_df.kb_oai_metadata_queried[0])

numpy.bool_

In [25]:
numpy.bool_

NameError: name 'numpy' is not defined

In [ ]:
# installation instruction not sufficient
# follow: https://github.com/aboSamoor/polyglot/issues/10
import polyglot

tt = """Onder den vorigen resident der Preanger- Regentschappen werd het denkbeeld aan de hand gedaan om uitzettingsmaatregelen toe te passen op Chineesche opiumsmokkelaars en op afnemers van gesmokkeld opium in dat gewest. De Regeering kon hierin niet treden, maar verzocht te dienen van advies omtrent de invoering der opiumregie in de Preanger- Regentschappen, met het oog op het toenemend verbruik van bedoeld artikel. Nu volgde een correspondentie tusschen den inmiddels benoemden nieuwen resident en den chef van den dienst der opiumregie, tengevolge waarvan het gevraagde advies luidde dat het ge westel yk bestu urshoofd geen bezwaar zag tegen de invoering der opiumregie in de Preanger-Regentschappen, mits het aan de inlandsche bevolking verboden bleef opium te koopen voor eigen gebruik. Eveneens werd overleg gepleegd met de militaire overheid te Tjimahi; de inlandsche soldaat schijnt niet zondar madat te kunnen bestaan; onder andere werd het gevoelen ingewonnen van den chef van den geneeskundigen dienst te genoemder plaatse."""




for sent in Text(tt).sentences:
    print(str(sent))

In [ ]:
num_errs = 0
for j, row in tqdm(df.iterrows()):
    try:
        Text(row["p"], hint_language_code="nl").sentences
    except Exception as e:
        num_errs += 1

In [ ]:
num_errs/df.shape[0]

In [ ]:
df_sents.memory_usage().sum()

In [ ]:
for j, row in tqdm(df.iterrows()):
    try:
        Text(row["p"], hint_language_code="nl").sentences
    except Exception as e:
        print(e)
        print(type(e))
        
        break